In [2]:
from pdfminer.high_level import extract_text
import os
import re
import pandas as pd
DATA_DIR = './Resume/data/'
resume_dfs = []
import string
import nltk
# nltk.download('stopwords')  // uncomment for the first time 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

def preprocessing(s):
    s = s.lower()
    s = re.sub(r"[^\x00-\x7F]+", "", s)   # removes unicode characters
    table = s.maketrans("","", string.punctuation)   
    s = s.translate(table)
    text_tokens = word_tokenize(s)
    
    tokens_without_sw = [word for word in text_tokens if not word in stopwords.words()]

    s = (" ").join(tokens_without_sw)
    
    return s



for job_type in os.listdir(DATA_DIR):
    resume_path = os.path.join(DATA_DIR, job_type)
    print(job_type)
    for resume in os.listdir(resume_path):
        text = extract_text(os.path.join(resume_path, resume))
        
        category_match = re.search(r'^(.+?)\n', text, re.MULTILINE)
        skills_match = re.search(r'skills\n(.+?)\n\n', text, re.DOTALL | re.IGNORECASE)
        education_match = re.search(r'Education(?: and Training|al Background)?\n(.+?)\n\n', text, re.DOTALL | re.IGNORECASE)

        category = job_type + ' '
        skills = ''
        education = ''
        id = resume[:-4]

        if category_match:
            category += category_match.group(1).strip()
        if skills_match:
            skills = skills_match.group(1).strip().replace('\n', ' ')
        if education_match:
            education = education_match.group(1).strip().replace('\n', ' ')
        
        category = preprocessing(category)
        skills = preprocessing(skills)
        education = preprocessing(education)

        resume_df = pd.DataFrame({'resume_id':[id],'Category': [category], 'Skills': [skills], 'Education': [education]})
        resume_dfs.append(resume_df)

resume_dfs = pd.concat(resume_dfs, ignore_index=True)

ACCOUNTANT
ADVOCATE
AGRICULTURE
APPAREL
ARTS
AUTOMOBILE
AVIATION
BANKING
BPO
BUSINESS-DEVELOPMENT
CHEF
CONSTRUCTION
CONSULTANT
DESIGNER
DIGITAL-MEDIA
ENGINEERING
FINANCE
FITNESS
HEALTHCARE
HR
INFORMATION-TECHNOLOGY
PUBLIC-RELATIONS
SALES
TEACHER


In [4]:
resume_dfs

,resume_id,Category,Skills,Education
0,10554236,accountant accountant,accounting general accounting accounts payable...,northern community college 1994 associate acco...
1,10674770,accountant staff accountant,accounting accounts payable accounts receivabl...,bachelor science accounting 2010 university no...
2,11163645,accountant accountant,accounts payables accounts receivables account...,computer applications specialist certificate p...
3,11759079,accountant senior accountant,accounting balance sheet budgets client client...,emory university goizueta business school 5 20...
4,12065211,accountant senior accountant,aderantcms excel quickbooks pro sql access pea...,bachelor business administration accounting au...
...,...,...,...,...
2479,86322251,teacher reading teacher,highly effective teacher motivator innovator s...,university south florida 2002 bachelor science...
2480,86597425,teacher history teacher,number wpm typing speed professional phone eti...,bachelor science interdisciplinary studiesbusi...
2481,90363254,teacher teacher,conflict resolution course development critica...,university phoenix 2007 master arts educationc...
2482,96547039,teacher teacher,cpr creative delivery aid policies problem sol...,conflict resolution techniques


In [3]:
resume_dfs.to_csv('resume_details.csv', index=False)